<a href="https://colab.research.google.com/github/Abdullah-Al-Arafat/CAP-5610-Machine-Learning/blob/master/ML_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CAP 5610: Machine Learning** 
Home work 2


In [1]:
import os
import numpy as np
from keras.datasets import cifar10
from keras import Sequential
from keras.utils import np_utils
from keras.optimizers import Adagrad
from keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Flatten, Dropout, BatchNormalization 
from keras import regularizers
from keras.models import model_from_json
from random import shuffle

Using TensorFlow backend.


Preparation/preprocessing of train, test, and validation dataset 

In [2]:
(train_images, train_labels),(test_images, test_labels) = cifar10.load_data()

## Creating validation set from training dataset

indices = np.random.permutation(len(train_images))

val_indices = indices[0:2000]
train_indices = indices[2000:]

val_images, val_labels = train_images[val_indices], train_labels[val_indices]

Train_images, Train_labels = train_images[train_indices], train_labels[train_indices]


## Normalizing the Train, test and validation sets

X_train = (Train_images/255).astype('float32')
y_train = np_utils.to_categorical(Train_labels, 10)

X_val = (val_images/255).astype('float32')
y_val = np_utils.to_categorical(val_labels, 10)

X_test = (test_images/255).astype('float32')
y_test = np_utils.to_categorical(test_labels, 10)

170500096/170498071 [==============================] - 22s 0us/step


In [29]:
X_train.shape

(48000, 32, 32, 3)

In [0]:
# Keras Parameters
batch_size = 32
nb_classes = 10
nb_epoch = 30
img_rows, img_col = 32, 32
img_channels = 3

## Weight Decay for regularizer
weight_decay = 1e-4

In [5]:

## Build the CNN model with keras

model = Sequential()
model.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation ='relu', input_shape =(img_rows, img_col, img_channels), data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(48, (3,3), strides = (1,1), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), strides= (1,1),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

#model.add(Conv2D(64, (3,3), strides= (2,2), padding='same', activation ='relu'))
#model.add(BatchNormalization())
 
model.add(Conv2D(128, (3,3), strides= (2,2), padding='same', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.1))


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(nb_classes, activation='softmax'))

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_4 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 48)        13872     
_________________________________________________________________
batch_normalization_5 (Batch (None, 16, 16, 48)        192       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 48)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 8, 64)          27712     
__________

In [0]:

# Initiate Adagrad/RMSprop optimizer
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [7]:
## Train the model and get the training and validation loss
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=10,
                    verbose=1, validation_data=(X_val, y_val))


Instructions for updating:
Use tf.cast instead.
Train on 48000 samples, validate on 2000 samples
Epoch 1/10
48000/48000 [==============================] - 29s 608us/step - loss: 1.3570 - acc: 0.5165 - val_loss: 1.2308 - val_acc: 0.5670
Epoch 2/10
48000/48000 [==============================] - 25s 517us/step - loss: 0.9666 - acc: 0.6595 - val_loss: 1.0247 - val_acc: 0.6465
Epoch 3/10
48000/48000 [==============================] - 25s 522us/step - loss: 0.8156 - acc: 0.7149 - val_loss: 1.1196 - val_acc: 0.5990
Epoch 4/10
48000/48000 [==============================] - 26s 536us/step - loss: 0.7095 - acc: 0.7536 - val_loss: 0.8739 - val_acc: 0.7110
Epoch 5/10
48000/48000 [==============================] - 26s 544us/step - loss: 0.6287 - acc: 0.7799 - val_loss: 0.9885 - val_acc: 0.6640
Epoch 6/10
48000/48000 [==============================] - 26s 544us/step - loss: 0.5625 - acc: 0.8030 - val_loss: 0.8295 - val_acc: 0.7170
Epoch 7/10
48000/48000 [==============================] - 26s 536us/s

In [8]:
## Score trained model.
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 160us/step
Test loss: 0.8835728015899659
Test accuracy: 0.7243
